In [1]:
import pandas as pd
df = pd.read_csv('/home/siddharth/Desktop/resume_parser/resume_pr/jobs_dataset_with_features.csv')

In [2]:
df.shape

(1615940, 2)

In [3]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [7]:
len(df['Role'].unique())

376

In [15]:
min_count = 6500
role_counts = df['Role'].value_counts()
dropped_classes = role_counts[role_counts < min_count].index
filtered_df = df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

len(filtered_df['Role'].unique())

61

In [12]:
filtered_df.shape

(520692, 2)

In [16]:
df  = filtered_df.sample(n=1000)

TFIDF

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [19]:
X = df['Features']
y = df['Role']

X_train , X_test , y_train , y_test  = train_test_split(X , y, test_size=0.2 , random_state = 42)

In [20]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [21]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_tfidf , y_train)

y_pred = rf_classifier.predict(X_test_tfidf)

print(accuracy_score(y_test , y_pred))

1.0


In [24]:
X_train

458621    4 to 15 Years Software Engineer BCA Proficienc...
183318    5 to 14 Years Supply Chain Manager BA Demand f...
209279    2 to 15 Years Landscape Architect M.Com Landsc...
24716     4 to 15 Years Project Manager B.Tech Project m...
439367    2 to 15 Years Customer Success Manager B.Tech ...
                                ...                        
119675    4 to 14 Years Investment Analyst BBA Investmen...
240624    4 to 10 Years Event Planner MCA Event planning...
182730    2 to 8 Years Administrative Assistant MCA Data...
227660    4 to 15 Years Social Media Coordinator B.Com C...
288884    2 to 10 Years Landscape Architect M.Com Landsc...
Name: Features, Length: 800, dtype: object

In [25]:
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)  
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText) 
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\]'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\]'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_15412/3739445999.py:3: SyntaxWarning: invalid escape sequence '\S'
  cleanText = re.sub('http\S+\s', ' ', txt)
/tmp/ipykernel_15412/3739445999.py:5: SyntaxWarning: invalid escape sequence '\S'
  cleanText = re.sub('#\S+\s', ' ', cleanText)
/tmp/ipykernel_15412/3739445999.py:6: SyntaxWarning: invalid escape sequence '\S'
  cleanText = re.sub('@\S+', '  ', cleanText)
/tmp/ipykernel_15412/3739445999.py:7: SyntaxWarning: invalid escape sequence '\]'
  cleanText = re.sub('[%s]' % re.es

In [27]:
def job_recommendation(resume_text):
    resume_text = cleanResume(resume_text)
    resume_tfidf = tfidf_vectorizer.transform([resume_text])
    predicted_category = rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [28]:
resume_file = """Objective:
Dedicated and results-oriented Banking professional with a strong background in financial analysis and customer service seeking opportunities to contribute to a reputable financial institution. Eager to leverage expertise in risk management, investment strategies, and relationship building to drive business growth and client satisfaction.

Education:
- Bachelor of Business Administration in Finance, XYZ University, GPA: 3.8/4.0
- Certified Financial Analyst (CFA) Level I Candidate

Skills:
- Proficient in financial modeling and analysis using Excel, Bloomberg Terminal, and other financial software
- Extensive knowledge of banking products and services, including loans, mortgages, and investment products
- Strong understanding of regulatory compliance and risk management practices in the banking industry
- Excellent communication and interpersonal skills, with a focus on building rapport with clients and colleagues
- Ability to work efficiently under pressure and adapt to changing market conditions

Experience:
Financial Analyst | ABC Bank
- Conducted financial analysis and risk assessment for corporate clients, including credit analysis, financial statement analysis, and cash flow modeling
- Developed customized financial solutions to meet clients' needs and objectives, resulting in increased revenue and client retention
- Collaborated with cross-functional teams to identify new business opportunities and optimize existing processes

Customer Service Representative | DEF Bank
- Provided exceptional customer service to bank clients, addressing inquiries, resolving issues, and promoting banking products and services
- Processed transactions accurately and efficiently, including deposits, withdrawals, and account transfers
- Educated customers on various banking products and services, helping them make informed financial decisions

Internship | GHI Investments
- Assisted portfolio managers with investment research and analysis, including industry and company-specific research, financial modeling, and performance analysis
- Prepared investment presentations and reports for clients, highlighting investment opportunities and performance metrics
- Conducted market research and analysis to identify trends and opportunities in the financial markets

Certifications:
- Certified Financial Planner (CFP)
- Series 7 and Series 63 Securities Licenses

Languages:
- English (Native)
- Spanish (Proficient)

"""
predicted_category = job_recommendation(resume_file)
print("Predicted Category:", predicted_category)

Predicted Category: Financial Analyst


In [30]:
resume_file = """
                Kshitij Agarwal
kshitijagarwal2808@gmail.com
github.com/kshitijagar
linkedin.com/in/kshitij-agar28
Roll No.: PES1UG21CS292
B.Tech (CSE) — PES University, Bangalore
Education
Degree/Certificate
B.Tech (Computer Science)
Senior Secondary
Secondary
Institute/Board
PES University
CBSE Board
CBSE Board
CGPA/Percentage
8.86 (Current)
94%
95.2%
Year
2021-Present
2021
2019
Skills
Programming: Python, C, Java, JavaScript, React.js, Node.js, React Native, Solidity Tools: Docker, GitHub,
Hadoop, Kaggle, Visual Studio Code Databases: MySQL, MongoDB, Firebase Operating Systems: Windows,
Linux
Key Courses
Mathematics: Linear Algebra, Statistics in Data Science, Data Analytics
Product Management: Software Engineering
Computer Science: Data Structures and Algorithms, Operating Systems, Computer Networking, Automata and Formal
Languages, Big Data, Information Security, Compiler Design, Blockchain
Experience
Summer Student - Ciena Corp
Jun 2024 - Aug 2024
• Collaborated with senior engineers to identify, troubleshoot, and resolve bugs in automation scripts for Packet Network
QA.
• Gained insights into networking protocols and operational mechanics of switches and routers.
• Contributed to automating logical port stats test cases, reducing test execution time by 93% (from 70 minutes to 5
minutes).
Projects
Blockchain Voting dApp
Mar 2024 - Apr 2024
• Developed a decentralized voting application using Solidity smart contracts and React.js frontend.
• Implemented secure voting mechanisms and blockchain integration for transparent and tamper-proof elections.
• Tech Stack: Solidity, React.js, Ethereum, Web3.js
MediTrack
Feb 2024 GitHub Link
• Developed a mobile app using Figma, React Native, and Firebase to track local disease spread.
• Integrated multiple APIs to provide real-time disease data.
• Tech Stack: Figma, React Native, Firebase
YKraft - Yet Another Kraft
Oct 2023 - Nov 2023 GitHub Link
• Developed an HTTP implementation of Kraft with essential functionalities.
• Enhanced system with metadata and log storage for efficient recovery from server failures.
• Tech Stack: Python, HTTP protocols
Sports Department Management System
Sept 2023 - Nov 2023 GitHub Link
• Backend: Developed a Sports Department Management System for the university using MySQL and Django providing
an intuitive interface.
• Integrated frontend using HTML and CSS to develop a Sports Department Management System.
• Tech Stack: MySQL, Django, HTML, CSS
Student Elective Manager
May 2023 - July 2023 Project Link
• Played a key role in making the app framework and front-end for improved user-friendliness.
• Contributed to the development of the project’s Firebase database and hosting the website on the web.
• Tech Stack: React.js, Firebase
SecureCipher: Data Encryption and Decryption Tool
Jan 2022 - Feb 2022 GitHub Link
• The project leverages a combination of Python libraries, advanced cryptographic techniques, and user interface (UI)
tools to create an intuitive and user-friendly Python application.
• Tech Stack: Python, Cryptography libraries, UI tools
Positions of Responsibility
Head of Events and Operations - IEEE CS Club, PES University
• Organized a guest lecture attended by over 400 participants.
• Led organization of a 24-hour hackathon with participants from vario
                """

predicted_category = job_recommendation(resume_file)
print("Predicted Category:", predicted_category)

Predicted Category: Backend Developer


In [29]:
import pickle
pickle.dump(rf_classifier,open('rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('tfidf_vectorizer_job_recommendation.pkl','wb'))